# STA 141B Project: Data Exploration
Our topic will be the Whitehouse petition API

In [118]:
from fastcache import clru_cache
import datetime
import requests
import numpy as np
import re
import pandas as pd

sandkey="TgltV3qLpU4EPb6LJF7Lx7wRJ6CMMVe5l7YBctBC"
prodkey="ETjU0uiiXFfA9AqvBUFooOEx2OmBdeq0nquzM1k4"

In [108]:
#Start building a function
@clru_cache(maxsize=128,typed=False)
def sandbox_test(limit="15"):
    base="https://api.whitehouse.gov/v1/petitions.json?limit="+limit
    request_get=requests.get(base+"&api_key="+prodkey)
    request_json=request_get.json()
    return(request_json)

In [83]:
tmp=sandbox_test("")

In [168]:
#UNIX time conversion code borrowed from stackoverflow
def extract_field(query):
    pet_id=[item["id"] for item in query]
    pet_created=[datetime.datetime.fromtimestamp(item["created"]).strftime('%Y-%m-%d %H:%M:%S') for item in query]
    pet_deadline=[datetime.datetime.fromtimestamp(item["deadline"]).strftime('%Y-%m-%d %H:%M:%S') for item in query]
    pet_title=[item["title"] for item in query]
    pet_issues=[", ".join([re.sub("amp;","",issue["name"]) for issue in item["issues"]]) for item in query]
    pet_type=[[category["name"] for category in item["petition_type"]] for item in query]
    pet_count=[item["signatureCount"] for item in query]
    pet_body=[re.compile("&(amp;)*#039;").sub("'",item["body"]).replace("\n","") for item in query]
    pet_url=[item["url"] for item in query]
    pet_list=[pet_id,pet_created,pet_deadline,pet_title,pet_issues,pet_type,pet_count,pet_body,pet_url]
    pet_array=np.array([np.array(column).reshape(100,1) for column in pet_list]).T.reshape(100,9)
    pet_df=pd.DataFrame(pet_array,columns=["id","created","deadline","title","issues","type","count","body","url"])
    return(pet_df)

In [169]:
extract_field(tmp["results"]).head()

,id,created,deadline,title,issues,type,count,body,url
0,2434791,2017-01-20 10:47:16,2017-02-19 10:47:16,Immediately release Donald Trump's full tax re...,"Foreign Policy, Government & Regulatory Reform",Tell us what the federal government is doing a...,1032520,The unprecedented economic conflicts of this a...,https://petitions.whitehouse.gov/petition/imme...
1,2434746,2017-01-20 10:38:48,2017-02-19 10:38:48,Divest or put in a blind trust all of the Pres...,Government & Regulatory Reform,Change an existing Administration policy,335608,In keeping with tradition and to avoid the app...,https://petitions.whitehouse.gov/petition/dive...
2,2436006,2017-01-20 12:57:47,2017-02-19 12:57:47,Let American Farmers Grow Hemp Once Again to C...,"Economy & Jobs, Government & Regulatory Reform...",Change an existing Administration policy,91810,Industrial hemp was once a dominant crop on th...,https://petitions.whitehouse.gov/petition/let-...
3,2434936,2017-01-20 11:00:04,2017-02-19 11:00:04,Donald Trump: Resign as President of the Unit...,"Civil Rights & Equality, Foreign Policy, Gover...",Propose a new Administration policy,127571,The President violates the Title of Nobility C...,https://petitions.whitehouse.gov/petition/dona...
4,2434701,2017-01-20 10:29:33,2017-02-19 10:29:33,Repeal the NFA,"Foreign Policy, Government & Regulatory Reform...",Change an existing Administration policy,245202,We the People wish to see the National Firearm...,https://petitions.whitehouse.gov/petition/repe...


In [171]:
petitions=extract_field(tmp["results"])
petitions.to_csv("petitions.csv")